In [13]:
import pandas as pd

pd.set_option('display.max_columns', 100)

In [23]:
test_df = pd.read_parquet("data/ml-engineer-main/iot_network_intrustion_dataset_test.parquet")
test_df = test_df.rename({"Unnamed: 0": "ID"}).set_index("ID")
test_df.head()

,Unnamed: 0,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,Fwd_Pkt_Len_Mean,Fwd_Pkt_Len_Std,Bwd_Pkt_Len_Max,Bwd_Pkt_Len_Min,Bwd_Pkt_Len_Mean,Bwd_Pkt_Len_Std,Flow_Byts/s,Flow_Pkts/s,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Tot,Fwd_IAT_Mean,Fwd_IAT_Std,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Tot,Bwd_IAT_Mean,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Len,Bwd_Header_Len,Fwd_Pkts/s,Bwd_Pkts/s,Pkt_Len_Min,Pkt_Len_Max,Pkt_Len_Mean,Pkt_Len_Std,Pkt_Len_Var,FIN_Flag_Cnt,SYN_Flag_Cnt,RST_Flag_Cnt,PSH_Flag_Cnt,ACK_Flag_Cnt,URG_Flag_Cnt,CWE_Flag_Count,ECE_Flag_Cnt,Down/Up_Ratio,Pkt_Size_Avg,Fwd_Seg_Size_Avg,Bwd_Seg_Size_Avg,Fwd_Byts/b_Avg,Fwd_Pkts/b_Avg,Fwd_Blk_Rate_Avg,Bwd_Byts/b_Avg,Bwd_Pkts/b_Avg,Bwd_Blk_Rate_Avg,Subflow_Fwd_Pkts,Subflow_Fwd_Byts,Subflow_Bwd_Pkts,Subflow_Bwd_Byts,Init_Fwd_Win_Byts,Init_Bwd_Win_Byts,Fwd_Act_Data_Pkts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,230773,163.152.127.193-192.168.0.13-10101-56361-17,192.168.0.13,56361,163.152.127.193,10101,17,20/08/2019 03:05:41 AM,141,2,1,1450.0,20.0,1430.0,20.0,725.0,997.020561,20.0,20.0,20.0,0.0,1.042553e+07,2.127660e+04,70.5,2.121320,72.0,69.0,69.0,69.0,0.0,69.0,69.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,0,16,8,14184.397163,7092.198582,20.0,1430.0,372.5,705.0,497025.0,0,0,0,0,0,0,0,0,0.0,496.666667,725.0,20.0,0,0,0,0,0,0,2,1450,1,20,-1,-1,2,0,0.0,0.0,0.0,0.0,70.5,2.121320,72.0,69.0,Anomaly,Mirai,Mirai-Hostbruteforceg
1,521223,192.168.0.24-210.89.164.90-64776-9988-17,192.168.0.24,64776,210.89.164.90,9988,17,25/07/2019 03:20:13 AM,1,1,1,32.0,32.0,32.0,32.0,32.0,0.000000,32.0,32.0,32.0,0.0,6.400000e+07,2.000000e+06,1.0,0.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,0,8,8,1000000.000000,1000000.000000,32.0,32.0,32.0,0.0,0.0,0,0,0,0,0,0,0,0,1.0,48.000000,32.0,32.0,0,0,0,0,0,0,1,32,1,32,-1,-1,1,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Anomaly,Mirai,Mirai-UDP Flooding
2,155309,192.168.0.13-192.168.0.16-9020-49784-6,192.168.0.13,9020,192.168.0.16,49784,6,20/05/2019 04:58:07 AM,482,0,5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,1.037344e+04,120.5,96.334487,265.0,72.0,0.0,0.0,0.0,0.0,0.0,482.0,120.5,96.334487,265.0,72.0,0,0,0,0,0,160,0.000000,10373.443983,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.000000,0.0,0.0,0,0,0,0,0,0,0,0,5,0,-1,32505,0,0,0.0,0.0,0.0,0.0,120.5,96.334487,265.0,72.0,Normal,Normal,Normal
3,42560,192.168.0.24-104.118.134.215-43238-443-6,104.118.134.215,443,192.168.0.24,43238,6,25/07/2019 03:21:48 AM,269,1,1,1441.0,1441.0,1441.0,1441.0,1441.0,0.000000,1441.0,1441.0,1441.0,0.0,1.071375e+07,7.434944e+03,269.0,0.000000,269.0,269.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0,0,0,0,32,32,3717.472119,3717.472119,1441.0,1441.0,1441.0,0.0,0.0,0,0,0,0,1,0,0,0,1.0,2161.500000,1441.0,1441.0,0,0,0,0,0,0,1,1441,1,1441,-1,252,1,0,0.0,0.0,0.0,0.0,269.0,0.000000,269.0,269.0,Anomaly,Mirai,Mirai-Ackflooding
4,326612,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11/07/2019 01:24:03 AM,73,0,2,0.0,2776.0,0.0,0.0,0.0,0.000000,1388.0,1388.0,1388.0,0.0,3.802740e+07,2.739726e+04,73.0,0.000000,73.0,73.0,0.0,0.0,0.0,0.0,0.0,73.0,73.0,0.000000,73.0,73.0,0,0,0,0,0,64,0.000000,27397.260274,1388.0,1388.0,1388.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,2082.000000,0.0,1388.0,0,0,0,0,0,0,0,0,2,2776,-1,1869,0,0,0.0,0.0,0.0,0.0,73.0,0.000000,73.0,73.0,Anomaly,Scan,Scan Port OS


In [12]:
test_df.describe()

,Unnamed: 0,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,...,Fwd_Act_Data_Pkts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
count,125157.000000,125157.000000,125157.000000,125157.000000,125157.000000,125157.000000,125157.000000,125157.000000,125157.000000,125157.000000,...,125157.000000,125157.0,125157.000000,125157.000000,125157.000000,125157.000000,125157.000000,125157.000000,125157.000000,125157.000000
mean,312324.453926,34950.321404,16366.327453,9.973881,650.039686,1.672132,1.477073,575.164681,939.124699,394.131059,...,1.504295,0.0,4.109658,0.476690,4.891984,3.713568,508.138405,57.585072,571.818013,469.042563
std,180705.616965,24730.698084,17534.091479,5.379128,3655.330974,4.283955,2.040416,1231.716717,2842.547976,620.525868,...,4.296188,0.0,73.238386,32.992821,123.149347,65.081136,2140.656964,1274.170555,2986.293031,1921.831800
min,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,156063.000000,9020.000000,8899.000000,6.000000,76.000000,0.000000,1.000000,0.000000,18.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,73.000000,0.000000,74.000000,71.000000
50%,311277.000000,51990.000000,9020.000000,6.000000,132.000000,1.000000,1.000000,32.000000,104.000000,30.000000,...,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,93.500000,0.000000,114.000000,78.000000
75%,468839.000000,56361.000000,10101.000000,17.000000,221.000000,2.000000,2.000000,1388.000000,1441.000000,1388.000000,...,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,140.000000,1.527525,154.000000,130.000000
max,625770.000000,65450.000000,65371.000000,17.000000,99839.000000,131.000000,560.000000,109846.000000,773284.000000,1460.000000,...,131.000000,0.0,9044.625000,8598.658250,26785.000000,6659.000000,92407.000000,66235.399301,95922.000000,92407.000000


In [10]:
train_df = pd.read_parquet("data/ml-engineer-main/iot_network_intrustion_dataset_test.parquet")
train_df.head()

,Unnamed: 0,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,230773,163.152.127.193-192.168.0.13-10101-56361-17,192.168.0.13,56361,163.152.127.193,10101,17,20/08/2019 03:05:41 AM,141,2,...,0.0,0.0,0.0,70.5,2.121320,72.0,69.0,Anomaly,Mirai,Mirai-Hostbruteforceg
1,521223,192.168.0.24-210.89.164.90-64776-9988-17,192.168.0.24,64776,210.89.164.90,9988,17,25/07/2019 03:20:13 AM,1,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Anomaly,Mirai,Mirai-UDP Flooding
2,155309,192.168.0.13-192.168.0.16-9020-49784-6,192.168.0.13,9020,192.168.0.16,49784,6,20/05/2019 04:58:07 AM,482,0,...,0.0,0.0,0.0,120.5,96.334487,265.0,72.0,Normal,Normal,Normal
3,42560,192.168.0.24-104.118.134.215-43238-443-6,104.118.134.215,443,192.168.0.24,43238,6,25/07/2019 03:21:48 AM,269,1,...,0.0,0.0,0.0,269.0,0.000000,269.0,269.0,Anomaly,Mirai,Mirai-Ackflooding
4,326612,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11/07/2019 01:24:03 AM,73,0,...,0.0,0.0,0.0,73.0,0.000000,73.0,73.0,Anomaly,Scan,Scan Port OS
